# Jeu de la vie

## installation de pycuda et import des packages

In [ ]:
!pip install pycuda

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

## Fonctions de bases 
- Définition de init_grid qui permet d'initialiser une grille $n\times  n$ aléatoirement. La graine est fixée à 0 pour des raisons de reproductibilité
- Définition de enlarge_grid qui permet d'élargir la grille et donc de s'abstenir de gérer les cas particulier des bords
- Définition de stat_alive qui permet de donner des statistiques sur le nombre de cellules vivantes dans une grille (élargie)

In [ ]:
def init_grid(n):
  # for parallel issues fix the same seed on each process
  np.random.seed(0)
  grid = np.random.randint(2,size=(n,n))
  return grid

In [ ]:
def enlarge_grid(grid):
  irange,jrange=grid.shape
  res_grid=np.zeros((irange+2,jrange+2))
  res_grid[1:irange+1,1:jrange+1]=grid[:,:]
  return res_grid

In [ ]:
def statalive(egrid):
    irange=egrid.shape[0]
    jrange=egrid.shape[1]
    if irange-2 <= 0 or jrange-2 <=0:
        return 0,0
    else:
        nalive=np.count_nonzero(egrid[1:irange,1:jrange])
        return nalive,100*nalive/((irange-2)*(jrange-2))


## Coeur de calcul : evolution
- Définition de evolution(grid) qui prend en entrée une grille élargie et retourne la grille résultantde l'application des règles du jeu de la vie à cette grille

In [ ]:
def evolution(grid):
  irange,jrange=grid.shape
  res_grid=np.zeros((irange,jrange))
  for i in range(1,irange-1):
      for j in range(1,jrange-1):
          nb_neigh=0
          #ligne dessus
          nb_neigh+=grid[i-1,j-1]+grid[i-1,j]+grid[i-1,j+1]
          #ligne courante
          nb_neigh+=grid[i,j-1]+grid[i,j+1]
          #ligne dessous
          nb_neigh+=grid[i+1,j-1]+grid[i+1,j]+grid[i+1,j+1]
          if nb_neigh==2:
              res_grid[i,j]=grid[i,j]
          elif nb_neigh==3:
              res_grid[i,j]=1
  return res_grid


## Fonction principale gamelife
- Définition de gamelife qui a partir d'une grille appel n itérations de evolution et affiche le résultat final

In [ ]:
def gamelife(grid,n):
    import matplotlib.pyplot as plt
    egrid=enlarge_grid(grid)
    for iter in range(n):
        egrid=evolution(egrid)
        nalive,percent=statalive(egrid)
    print("{0} cellules vivantes soit {1:.2f} %".format(nalive,percent))
    plt.figure()
    plt.imshow(egrid[1:egrid.shape[0]-1,1:egrid.shape[1]-1])
    

## Création d'une grille $32\times 32$

In [ ]:
grid=init_grid(32)

## Appel à gamelife sur 10 itérations

In [ ]:
gamelife(grid,10)

## Définition du kernel conway qui calcule evolution sur gpu à compléter

In [ ]:
kernels = SourceModule("""
__global__ void conway(int *res_grid,int *input_grid)
{
   /* /!\ Attention res_grid et input_grid sont des grilles elargies
      Par consequent il faut penser a decaler l'indice
      /!\ ATTENTION 2 le bloc de thread est dimensionne a la taille de la grille
          initiale non elargie, encore une fois il faut penser egalement aux 
          indices
   */
   // position ix,iy de la cellule
   int ix = TODO;
   int iy = TODO;
   int neighbs = 0;
   // ligne du dessus
   neighbs += TODO ;
   // ligne courante
   neighbs += TODO ;
   // ligne du dessous
   neighbs += TODO ;
   if(neighbs < 2 || neighbs > 3)
   {
      TODO;
   }
   else
   {
      if(neighbs == 3)
      {
         TODO;
      }
      if(neighbs == 2)
      {
         TODO;
      }
   }              
}
""")

##Définition de g_gamelife qui doit appliquer le kernel conway à compléter 

In [ ]:
def g_gamelife(grid,n):
    import matplotlib.pyplot as plt
    xdim,ydim = grid.shape
    egrid     = enlarge_grid(grid)
    egrid     = egrid.astype(np.int32)
    g_input   = cuda.mem_alloc(egrid.nbytes)
    g_output  = cuda.mem_alloc(egrid.nbytes)
    gconway   = kernels.get_function("conway")
    for iter in range(n):
        cuda.memcpy_htod(g_input,egrid)
        cuda.memcpy_htod(g_output,egrid)
        # TODO
        gconway(g_output,g_input,block=(...,...,1))
        cuda.memcpy_dtoh(egrid,g_output)
    nalive,percent = statalive(egrid)
    print(f"{nalive} cellules vivantes soit {percent:.2f} %")
    g_input.free()
    g_output.free()
    plt.figure()
    plt.imshow(egrid)

## Appel à gamelife sur 10 itérations

In [ ]:
g_gamelife(grid,10)